## Question 1

In [3]:
from fonction import *

df_academie = pd.read_csv("academies_virgule.csv")
df_academie

,academie,wikipedia,vacances,dep,departement,region
0,Académie de Lyon,fr:Académie de Lyon (éducation),Zone A,01,Ain,Auvergne-Rhône-Alpes
1,Académie d'Amiens,fr:Académie d'Amiens (éducation),Zone B,02,Aisne,Nord-Pas-de-Calais-Picardie
2,Académie de Clermont-Ferrand,fr:Académie de Clermont-Ferrand,Zone A,03,Allier,Auvergne-Rhône-Alpes
3,Académie d'Aix-Marseille,fr:Académie d'Aix-Marseille,Zone B,04,Alpes-de-Haute-Provence,Provence-Alpes-Côte d'Azur
4,Académie d'Aix-Marseille,fr:Académie d'Aix-Marseille,Zone B,05,Hautes-Alpes,Provence-Alpes-Côte d'Azur
...,...,...,...,...,...,...
96,Académie de Versailles,fr:Académie de Versailles (éducation),Zone C,95,Val-d'Oise,Île-de-France
97,Académie de la Guadeloupe,fr:Académie de la Guadeloupe,NaN,971,Guadeloupe,Guadeloupe
98,Académie de la Martinique,fr:Académie de la Martinique,NaN,972,Martinique,Martinique
99,Académie de Guyane,fr:Académie de la Guyane,NaN,973,Guyane,Guyane


In [5]:
table_academie = dataframe_to_table(df_academie)
table_academie

In [8]:
parquet_file = 'academie.parquet'
table_to_parquet(table_academie, parquet_file)

In [9]:
parquet_to_table('academie.parquet')

pyarrow.Table
academie: string
wikipedia: string
vacances: string
dep: string
departement: string
region: string
----
academie: [["Académie de Lyon","Académie d'Amiens","Académie de Clermont-Ferrand","Académie d'Aix-Marseille","Académie d'Aix-Marseille",...,"Académie de Versailles","Académie de la Guadeloupe","Académie de la Martinique","Académie de Guyane","Académie de la Réunion"]]
wikipedia: [["fr:Académie de Lyon (éducation)","fr:Académie d'Amiens (éducation)","fr:Académie de Clermont-Ferrand","fr:Académie d'Aix-Marseille","fr:Académie d'Aix-Marseille",...,"fr:Académie de Versailles (éducation)","fr:Académie de la Guadeloupe","fr:Académie de la Martinique","fr:Académie de la Guyane","fr:Académie de La Réunion"]]
vacances: [["Zone A","Zone B","Zone A","Zone B","Zone B",...,"Zone C",null,null,null,null]]
dep: [["01","02","03","04","05",...,"95","971","972","973","974"]]
departement: [["Ain","Aisne","Allier","Alpes-de-Haute-Provence","Hautes-Alpes",...,"Val-d'Oise","Guadeloupe","Marti